In [ ]:
# publication name, date, byline, section and  headlines

In [4]:
#crawl ['title', 'author', 'time', 'section', 'byline_list', 'url']
## Got Ban From each website , robot detection


import re
import requests
import json
import os


def get_html(url):
    try:
        html = getHTMLText(url)
        return html
    except:
        print("failed connect")


def getHTMLText(url):
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ''
    
def crawl_sitemap(url):
    #download sitemap file
    sitemap = get_html(url)
    #extract the sitemap links
    links = re.findall('<loc>(.*?)</loc>',sitemap)
    
    return links
    
def reHtml(htmlContent):
    # title
    title = re.findall(r'<h1 class="lede-text-v2__hed">(.*?)</h1>', htmlContent, re.DOTALL)[0].strip()

    # author
    author = re.findall(r'<a class="author-v2__byline".*?>(.*?)</a>', htmlContent, re.DOTALL)[0].strip()

    # time
    time = re.findall(r'<time.*?<noscript>(.*?)</noscript>.*?</time>', htmlContent, re.DOTALL)[0].strip()

    # section
    section = re.findall(r'<div class="eyebrow-v2">(.*?)</div>', htmlContent, re.DOTALL)[0].strip()

    # byline
    byline_list = re.findall(r'<div class="abstract-v2__item-text">(.*?)</div>', htmlContent, re.DOTALL)
    
    return title, author, time, section, byline_list

# Date
def judgeDate(time):
    day = int(re.findall(r'March (\d\d), 2020', time)[0])
    if 18 < day < 26:
        return True
    else:
        return False

def generateData(links):
    '''
    {'NewsTitle': {'title': NewsTitle, 'author': James, 
    'time': 'March 15, 2020, 6:57 PM EDT', 
    'section': 'Business', 'byline_list': ['l1', 'l2']}}
    '''
    data = {}
    keys = ['title', 'author', 'time', 'section', 'byline_list', 'url', 'The']
    
    for link in links:
        htmlContent = get_html(link) 
        print(htmlContent)
        title, author, time, section, byline_list = reHtml(htmlContent)
        info = {}
        info['title'], info['author'], info['time'], info['section'], info['byline_list'], info['url'] = title, author, time, section, byline_list, link
        data[title] = info
    return data

if __name__ == '__main__':
    file1 = open('urls.txt','a')
    file2 = open('htmls.txt','a')
    
    links = crawl_sitemap('https://www.bloomberg.com/feeds/bbiz/sitemap_news.xml')
    
    data = generateData(links)
    
    with open('data.txt', 'w') as outfile:
        json.dump(data, outfile)


<!doctype html>
<html>
<head>
    <title>Bloomberg - Are you a robot?</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" type="text/css" href="https://assets.bwbx.io/font-service/css/BWHaasGrotesk-55Roman-Web,BWHaasGrotesk-75Bold-Web,BW%20Haas%20Text%20Mono%20A-55%20Roman/font-face.css">
    <style rel="stylesheet" type="text/css">
        html, body, div, span, applet, object, iframe,
        h1, h2, h3, h4, h5, h6, p, blockquote, pre,
        a, abbr, acronym, address, big, cite, code,
        del, dfn, em, img, ins, kbd, q, s, samp,
        small, strike, strong, sub, sup, tt, var,
        b, u, i, center,
        dl, dt, dd, ol, ul, li,
        fieldset, form, label, legend,
        table, caption, tbody, tfoot, thead, tr, th, td,
        article, aside, canvas, details, embed,
        figure, figcaption, footer, header, hgroup,
        menu, nav, output, ruby, section, summary,
        time, mark, audio, video {
            

IndexError: list index out of range

In [96]:
#crawl only headlines

def crawl_sitemap(url):
    #download sitemap file
    sitemap = get_html(url)
    #extract the sitemap links
    links = re.findall('<loc>(.*?)</loc>',sitemap)
    titles = re.findall('<news:title>(.*?)</news:title>',sitemap)
    times = re.findall('<news:publication_date>(.*?)<news:publication_date>', sitemap)
    return links, titles, times
    

if __name__ == '__main__':
    links_BB, titles_BB, times_BB = crawl_sitemap('https://www.bloomberg.com/feeds/bbiz/sitemap_news.xml')
    links_WSJ, titles_WSJ, times_WSJ = crawl_sitemap('https://www.wsj.com/wsjsitemaps/wsj_google_news.xml')
    links_FT, titles_FT, times_FT = crawl_sitemap('https://www.ft.com/sitemaps/news.xml')
    links = links_BB + links_WSJ + links_FT
    titles = titles_BB + titles_WSJ + titles_FT
    times = times_BB + times_WSJ + times_FT
    

In [114]:
# Prepocessing text
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
ignore_word = set(['apos','With', 'Says', 'Bloomberg', 'Day', 'Be', 'Is', 'To', 'The', 'Letter', 'business', 'How'])
#use for word search for title
dic = {}
#use for raw text data
total_words = []
for s in titles:
    #filter the content in () ex. (article) (radio)/ (podcast)
    s = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", "", s)
    #tokenize
    raw_words = nltk.word_tokenize(s)
    wordnet_lematizer = WordNetLemmatizer()
    words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
    #filter
    filtered_words = [word.lower() for word in words if word not in stopwords.words('english') and word.isalpha() and word not in ignore_word]
    total_words += [word for word in filtered_words]
    filtered_words_set = {word for word in filtered_words}
    dic[s] = filtered_words_set

[nltk_data] Downloading package punkt to /Users/siyuqiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/siyuqiu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siyuqiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
#most frequent word daily
from nltk import FreqDist
freq_dist = FreqDist(total_words) 
most_common_freq = freq_dist.most_common(10)
most_common = [word[0] for word in most_common_freq]
print(most_common_freq)

[('coronavirus', 59), ('virus', 29), ('lockdown', 22), ('us', 19), ('new', 15), ('uk', 14), ('oil', 13), ('crisis', 12), ('pandemic', 12), ('trump', 11)]


In [116]:
#titles which contain most frequent word
common_word_title_dic = {}
for word in most_common:
    common_word_title_dic[word] = []
    for k in dic:
        if word in dic[k]:
            common_word_title_dic[word].append(k)
print(common_word_title_dic)

{'coronavirus': ['Dorsey Pledges $1 Billion to Coronavirus Fight: Donation Tracker', 'Metzl Says AI Will Aid in Fighting Coronavirus ', 'U.K. Reports Its Deadliest Day With 786 Coronavirus Deaths', 'When and How Will the Coronavirus Pandemic End?', 'Coronavirus Blew Up Your Forecast – Now What?', 'Counting the Costs of the Coronavirus Contagion in the Middle East', 'Coronavirus latest: Trump says US to put hold on its WHO funding', 'Donald Trump removes watchdog tapped to oversee coronavirus relief', 'EU science chief resigns with blast at coronavirus response', 'Asos raises £240m in share placing to deal with coronavirus', 'Coronavirus tracked: the latest figures as the pandemic spreads | Free to read', 'Tesco urged to pay dividend despite coronavirus', 'FCA plan focuses on consumer protection long after coronavirus', 'Deaths from coronavirus far higher in England than first reported', 'Flexible offices could yet return to favour after coronavirus\xa0', 'Coronavirus: remedies for a he